# Finding MLE and MAP on simple models

The example below I use from: https://pyro.ai/examples/svi_part_i.html#A-simple-example

We will start with the quote from: https://forum.pyro.ai/t/correct-map-guide-without-using-automatic-guide-generation/940 

**"If you want a MAP estimate for alpha, you need to pyro.sample it from a prior distribution in your model and delta distribution in your guide just like your second guide does with theta, rather than treating it as a constant. If you instead want a maximum likelihood estimate of alpha, you need to wrap it with a pyro.param call in the model and omit it from your guide"**


In [1]:
# General
import os
from collections import defaultdict
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
# Pytorch
import torch
from torch.distributions import constraints
import torchvision.datasets as dset
import torch.nn as nn
import torchvision.transforms as transforms
# Pyro
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
pyro.enable_validation(True)    # <---- This is always a good idea!

Let's define our data and train function

In [2]:
data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))

In [3]:
def train(model, guide,lr=0.005):
  # set up the optimizer
  adam_params = {"lr": lr, "betas": (0.90, 0.999)}
  optimizer = Adam(adam_params)
  pyro.clear_param_store()
  # setup the inference algorithm
  svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

  n_steps = 5000
  # do gradient steps
  for step in range(n_steps):
      loss = svi.step(torch.tensor(data))
      if step % 1000 == 0:
          print('%.4f' %(loss))

## MLE


In [4]:
def model_MLE(data):
    # define the hyperparameters that control the beta prior

    # sample f from the beta prior
    f = pyro.param("latent_fairness", torch.tensor(0.1,requires_grad=False), constraint=constraints.positive)
    # loop over the observed data
    with pyro.plate('data', len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

In [5]:
def guide_MLE(data):
  # mu_map = pyro.param('mu_map', torch.tensor(1.0))
  # pyro.sample('theta', dist.Delta(mu_map))
  return None

In [6]:
train(model_MLE,guide_MLE)

14.2370
6.7301
6.7301
6.7301
6.7301


In [7]:
pyro.param('latent_fairness').item()

0.5999999642372131

0.599999 very close to 0.6. 

## MAP

### Through Posterior

This is nothing special, I copy whole from the tutorial. Prior is Beta(10,10), then the posterior is 0.533333

In [8]:
def model_posterior(data):
    # define the hyperparameters that control the beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    with pyro.plate('data', len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_posterior(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

In [9]:
train(model_posterior,guide_posterior,lr=0.0005)

6.7590
7.0591
7.0463
7.0819
7.0703


In [10]:
import math
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))


based on the data and our prior belief, the fairness of the coin is 0.530 +- 0.090


MAP is the mode of posterior distribution, now or we plot the distribution, or we compute it directly. I choose latter



In [11]:
import math
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()
print((alpha_q-1)/(alpha_q+beta_q-2))

0.5318537663134782


### Through using Delta distribution

When define Delta distribution, don't init it as 1 for Beta, it will jump to NaN or inf.

In [12]:
def model_map(data):
    f = pyro.sample("latent_fairness", dist.Beta(1, 1))
    with pyro.plate('data', len(data)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_map(data):
    mu_map = pyro.param("mu_map", torch.tensor(0.5),
                         constraint=constraints.positive)
    return pyro.sample("latent_fairness", dist.Delta(mu_map))

In [13]:
train(model_map,guide_map)

6.9315
6.7301
6.7301
6.7301
6.7301


In [14]:
map_final = guide_map(data)

In [15]:
map_final

tensor(0.6000, grad_fn=<ExpandBackward>)

Ok beta(1,1) is uniform, then map equal to mle. = 0.6 

In [16]:
def model_map(data):
    f = pyro.sample("latent_fairness", dist.Beta(10, 10))
    with pyro.plate('data', len(data)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_map(data):
    mu_map = pyro.param("mu_map", torch.tensor(0.5),
                         constraint=constraints.positive)
    return pyro.sample("latent_fairness", dist.Delta(mu_map))

In [17]:
train(model_map,guide_map)
map_final = guide_map(data)

5.6719
5.6004
5.6004
5.6004
5.6004


In [18]:
map_final

tensor(0.5357, grad_fn=<ExpandBackward>)

MAP of Beta(10,10) is same as above, great.

What is the real MAP if we compute using our knowledge?

In [19]:
new_alpha = 10 + 6
new_beta = 10 + 4
print((new_alpha-1)/(new_alpha+new_beta-2))

0.5357142857142857


You see, using Delta this time is more accurate than compute whole posterios distribution. Very accurate indeed.